In [1]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import re

# function created for this project (should create link if I ever share this)
from validate_date import validate_date

def loan_amortization(principal, interest_rate, start_date, end_date, periodtype):
    
    """ 
        Args:
        principal (float): The principal amount of the loan.
        interest_rate (float): The annual interest rate, expressed as a percent (e.g. 5% = 5).
        start_date (str or datetime): The start date of the loan in 'YYYY-MM-DD' format or as a datetime object.
        end_date (str or datetime): The end date of the loan in 'YYYY-MM-DD' format or as a datetime object.
        periodtype (str): The type of period to use for the loan payments, which can be one of 'D' (days), 'W' (weeks), 'BW' (biweekly), 'M' (months), or 'Y' (years).

    Returns:
        pandas.DataFrame: A DataFrame containing the amortization schedule for the loan, with columns for the payment number, payment date, payment amount, interest paid, principal paid, and balance.
    """
    
    if validate_date(start_date,end_date) == False:
        raise TypeError("start_date and end_date must be a string in 'YYYY-MM-DD' format or a datetime object")
    
    # if the date is in the string format, convert it
    if not isinstance(start_date, datetime):
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
    if not isinstance(end_date, datetime):
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
                        
    # input type checking for principal and interest_rate
    if not isinstance(principal, (int, float)):
        raise TypeError("Principal amount should be numeric (int or float)")
    if not isinstance(interest_rate, (int, float)):
        raise TypeError("Interest rate should be numeric and in % (int or float)")
        
    # period-type definition 
    if periodtype == 'D':
        periods = int((end_date - start_date).days)
        days_between_payments = 1
        adjusted_rate = interest_rate / 36500
    elif periodtype == 'W':
        periods = int((end_date - start_date).days / 7)
        days_between_payments = 7
        adjusted_rate = interest_rate / 5200
    elif periodtype == 'BW':
        periods = int((end_date - start_date).days / 14)
        days_between_payments = 14
        adjusted_rate = interest_rate / 2600
    elif periodtype == 'M':
        periods = int((end_date.year - start_date.year) * 12 + (end_date.month - start_date.month))
        days_between_payments = int(relativedelta(months=1).days)
        adjusted_rate = interest_rate / 1200
    elif periodtype == 'Y':
        periods = int(end_date.year - start_date.year)
        days_between_payments = int(relativedelta(years=1).days)
        adjusted_rate = interest_rate 
    else:
        raise TypeError("periodtype should be one of the following: 'D', 'W', 'BW', 'M', 'Y'")
    
    monthly_payment = principal * adjusted_rate / (1 - (1 + adjusted_rate) ** (-periods))

    # create a list of dates for each payment
    payment_dates = [start_date + relativedelta(days=(days_between_payments * i)) for i in range(periods)]

    # lists for the payment number, payment amount, interest, principal, and balance
    payment_number = list(range(1, periods + 1))
    payment_amount = monthly_payment
    interest = []
    principal_paid = []
    balance = [principal]

    # interest, principal, and balance for each payment period
    for i in range(periods):
        interest.append(round(balance[i] * adjusted_rate, 2))
        principal_paid.append(round(monthly_payment - interest[i], 2))
        balance.append(round(balance[i] - principal_paid[i], 2))

    # make the amortization-schedule dataframe
    data = {
        'Payment Number': payment_number,
        'Payment Date': payment_dates,
        'Payment Amount': payment_amount,
        'Interest Paid': interest,
        'Principal Paid': principal_paid,
        'Balance': balance[:-1]
    }
    df = pd.DataFrame(data)

    # set the index to the payment date
    df.set_index('Payment Date', inplace=True)


    # Truncate the DataFrame to the end date
    df = df.truncate(before=start_date, after=end_date)

    # apply formating for dollar signs and two decimals
    df.index.name = 'Date'
    df['Payment Amount'] = df['Payment Amount'].apply(lambda x: '${:,.2f}'.format(x))
    df['Interest Paid'] = df['Interest Paid'].apply(lambda x: '${:,.2f}'.format(x))
    df['Principal Paid'] = df['Principal Paid'].apply(lambda x: '${:,.2f}'.format(x))
    df['Balance'] = df['Balance'].apply(lambda x: '${:,.2f}'.format(x))

    return df

In [2]:
# Test: 5-year loan, starting on January 1, 2023 and ending on December 31, 2027
schedule = loan_amortization(100000, 5, "2023-1-1", "2027-12-31", 'W')

schedule

,Payment Number,Payment Amount,Interest Paid,Principal Paid,Balance
Date,,,,,
2023-01-01,1,$434.88,$96.15,$338.73,"$100,000.00"
2023-01-08,2,$434.88,$95.83,$339.05,"$99,661.27"
2023-01-15,3,$434.88,$95.50,$339.38,"$99,322.22"
2023-01-22,4,$434.88,$95.18,$339.70,"$98,982.84"
2023-01-29,5,$434.88,$94.85,$340.03,"$98,643.14"
...,...,...,...,...,...
2027-11-21,256,$434.88,$2.08,$432.80,"$2,167.39"
2027-11-28,257,$434.88,$1.67,$433.21,"$1,734.59"
2027-12-05,258,$434.88,$1.25,$433.63,"$1,301.38"


In [3]:
import os
os.getcwd()

'C:\\Users\\dbawa\\ZZ__School Related\\Other\\Personal Projects\\Personal Projects\\Amortization Table'